In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix, vstack, hstack, save_npz, load_npz, block_diag, identity, random
from scipy.sparse.linalg import inv, spsolve, splu
from scipy.linalg import lu
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool, shared_memory
import matplotlib.pyplot as plt
import time
import cProfile
import pstats 
import csv
import os 
from tqdm import tqdm

In [2]:
def lower_block_bidiagonal_nonsingular(n_blocks, block_size):
    """
    Generate a nonsingular sparse lower block bidiagonal matrix in CSR format.

    Parameters:
        n_blocks (int): Number of diagonal blocks.
        block_size (int): Size of each square block.

    Returns:
        scipy.sparse.csr_matrix: The resulting nonsingular sparse matrix.
        numpy.ndarray: The corresponding RHS vector.
    """
    N = n_blocks * block_size  # Total size of the matrix
    data, row_indices, col_indices = [], [], []

    # Generate diagonal (B_i) and lower diagonal (L_i) blocks
    for i in range(n_blocks):
        row_offset = i * block_size
        col_offset = i * block_size

        # Ensure nonzero entries in the main diagonal block (B_i)
        block_main = np.random.rand(block_size, block_size) + np.eye(block_size)  # Make B_i non-singular
        for r in range(block_size):
            for c in range(block_size):
                val = block_main[r, c]
                data.append(val)
                row_indices.append(row_offset + r)
                col_indices.append(col_offset + c)

        # Lower block (L_i), ensuring nonzero entries
        if i < n_blocks - 1:
            row_offset = (i + 1) * block_size
            col_offset = i * block_size
            block_lower = np.random.rand(block_size, block_size)  # Random values ensure nonzero entries

            for r in range(block_size):
                for c in range(block_size):
                    val = block_lower[r, c]
                    data.append(val)
                    row_indices.append(row_offset + r)
                    col_indices.append(col_offset + c)

    # Create sparse CSR matrix
    sparse_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(N, N))

    # Generate a random RHS vector (column vector)
    rhs_vector = np.random.rand(N, 1)  # Nx1 dense vector

    return sparse_matrix, rhs_vector

In [57]:
p = 4 # no. of processors
k = 2 # recursion/iteration depth
n = int(p*2**k)

n_blocks = n + 1
block_size = 2

# M, f = lower_block_bidiagonal_nonsingular(n_blocks, block_size)
# x = spsolve(M,f)

# save_folder = "LBBM_p4" # Lower block bidiagonal matrix for 4 processors
# save_npz(f"{save_folder}/n_{n_blocks}_mat.npz",M)
# np.save(f"{save_folder}/n_{n_blocks}_rhs.npy",f)
# np.save(f"{save_folder}/n_{n_blocks}_sol.npy",x)

In [58]:
save_folder = "LBBM_p4"
M,f,x = load_npz(f"{save_folder}/n_{n_blocks}_mat.npz"), np.load(f"{save_folder}/n_{n_blocks}_rhs.npy"), np.load(f"{save_folder}/n_{n_blocks}_sol.npy")

In [ ]:
def placeholder_name(M, f, block_size : int, processors : int):
    """ 
    Performs Block Cyclic Reduction (BCR) in parallel for solving lower block bidiagonal systems.

    Parameters:
    -----------
    M : scipy.sparse.csr_matrix or numpy.ndarray
        The coefficient matrix of size (N, N), where N = (n+1) * block_size.
        Must be a square lower block bidiagonal matrix.

    f : numpy.ndarray
        The right-hand side (RHS) vector of size (N, 1), corresponding to Mx = f.

    block_size : int
        The size of each block in the matrix.

    processors : int
        The number of processors used for parallel block cyclic reduction.

    Returns:
    --------
    x : numpy.ndarray
        The solution vector of size (N, 1) satisfying Mx = f.
    """
    N, L = M.shape
    assert N == L,  f"M must be sqaure but has dimensions {N}x{L}"
    n = (N - 1) // block_size
    assert n % processors == 0, f"M must have size (n+1)*block_size x (n+1)*block_size, where n = p * 2**k. p is not a multiple of n."
    nbyp = n // processors 
    assert ((nbyp & (nbyp-1) == 0) and nbyp != 0), f"M must have size (n+1)*block_size x (n+1)*block_size, where n = p * 2**k. n/p is not a power of two."

    row_index_start = block_size
    row_index_end = block_size*(1+nbyp)
    col_index_start = 0
    col_index_end = block_size*(nbyp+1)
    
    # Divide among the processors
    M_k_list = []
    f_k_list = []
    B_k_s_list = []
    A_k_s_list = []
    f_k_s_list = []
    
    for _ in range(processors):
        # Perform the forward step
        M_copy = M[row_index_start:row_index_end, col_index_start:col_index_end]
        f_copy = f[row_index_start:row_index_end]
        M_k, f_k, B_k_s, A_k_s, f_k_s = forward_placeholder(M_copy, f_copy, block_size, processors, [], [], [])

        # Store the results for inter-processor communication
        M_k_list.append(M_k)
        f_k_list.append(f_k)

        # Store the results for the backward step
        B_k_s_list.append(B_k_s)
        A_k_s_list.append(A_k_s)
        f_k_s_list.append(f_k_s)

        # Update the indices for the next processor
        row_index_start = row_index_end
        row_index_end += nbyp*block_size 
        col_index_start = col_index_end - block_size
        col_index_end += block_size*nbyp
    
    x0 = spsolve(M[:block_size,:block_size],f[:block_size]) # The master of all processors
    base_case_x = [x0]

    # Only serial part of the algorithm
    for i in range(processors):
        B_k = M_k_list[i][:,:block_size]
        A_k = M_k_list[i][:,block_size:]
        f_k = f_k_list[i]
        x_next = spsolve(A_k,f_k.flatten()-B_k@base_case_x[i])
        base_case_x.append(x_next)
    
    # Perform the backward step
    for i in range(processors):
        B_k_s = B_k_s_list[i] 
        A_k_s = A_k_s_list[i] 
        f_k_s = f_k_s_list[i] 

        some_res = backward_placeholder(B_k_s, A_k_s, f_k_s, base_case_x[i:i+2], block_size, processors)
        #print(some_res)
        break
    
    
def forward_placeholder(M, f, block_size : int, processors : int, B_s = [], A_s = [], f_s = []):
    n,m = M.shape
    if n == block_size:
        print(f"Forward step finished!")
        return M,f,B_s,A_s,f_s
    
    M_next = csr_matrix((n//2,n//2+block_size))
    f_next = np.zeros(n//2)
    # Do one step
    for i in range(0,n,2*block_size):
        # Extract block elements from input
        B1 = M[i:i+block_size, i:i+block_size] 
        A1 = M[i:i+block_size, i + block_size: i + 2*block_size] 
        B2 = M[i + block_size: i + 2*block_size,i + block_size: i + 2*block_size ]
        A2 = M[i + block_size: i + 2*block_size,i + 2*block_size: i + 3*block_size]
        f1 = f[i:i+block_size]
        f2 = f[i + block_size: i + 2*block_size]

        # Store the values for the backward step
        B_s.append(B1)
        A_s.append(A1)
        f_s.append(f1)

        # Compute inverses and values for the next depth. This is equivalent to removing all odd indices from the input
        B2_inv = inv(B2)
        A1_inv = inv(A1)
        new_B1 = A1_inv@B1
        new_A1 = -B2_inv@A2
        new_f1 = A1_inv@f1 - B2_inv@f2

        # B_s.append(new_B1)
        # A_s.append(new_A1)
        # f_s.append(new_f1)

        # Set the new values to obtain a reduced system of half the original size
        j = i//2
        M_next[j:j+block_size,j:j+block_size] = new_B1
        M_next[j:j+block_size,j+block_size:j+2*block_size] = new_A1
        f_next[j:j+block_size] = new_f1.flatten()

    # Recursively apply the same procedure
    return forward_placeholder(M_next,f_next,block_size,processors,B_s,A_s,f_s)

def backward_placeholder(B_s, A_s, f_s, x_s, block_size : int, processors : int):
    x_result = np.concatenate((x_s[0],x_s[1]))
    x_next = x_s[0]

    for i in range(len(B_s)-1,-1,-1):
        B = B_s[i]
        A = A_s[i]
        f = f_s[i].flatten()
        x_next = spsolve(A,f - B@x_next)
        x_result = np.concatenate((x_result, x_next))
    print(x_result.flatten())
    return x_result.flatten()
    

np.set_printoptions(precision=8, suppress=True)
placeholder_name(M,f,block_size=block_size,processors=p)
print(x)
# selected_numbers = zip(x[0::4], x[1::4])
# selected_numbers = np.array(list(selected_numbers)).flatten()
# print(selected_numbers)


Forward step finished!
Forward step finished!
Forward step finished!
Forward step finished!
[ 0.48647425 -0.20084005 -0.21812064  0.65115813 -0.08274303  0.26345255
 -0.16983722  0.30626582 -0.03114629  0.28064602]
[ 0.48647425 -0.20084005  0.23309244  0.00811299 -0.08274303  0.26345255
 -0.16983722  0.30626582 -0.21812064  0.65115813  0.88325422 -0.07304672
  0.48174632 -0.1657839   0.43910117  0.11724197  0.23124714 -0.56470413
  0.92287657  0.03407044  0.10228898  0.46162783  0.06601929  0.23358886
 -0.04730871  0.39700067  0.30031906  0.40821593  0.22053257 -0.22680817
  0.12370639  0.02582541  0.43580205 -0.234097  ]


In [114]:
abba = [1,2,3,4,5,6,7,8]

for i in range(len(abba)-1):
    print(abba[i:i+2])

[1, 2]
[2, 3]
[3, 4]
[4, 5]
[5, 6]
[6, 7]
[7, 8]


## About this repository

This repository attempts to implement parallel cyclic reduction for the solution of block tridiagonal linear systems. In other words, solving $Mx=f$ faaast. This is a work in progress and only some of the code is stable. 

The first implementation reconstructed the algorithm proposed by P, Amodio and N. Mastronardi in [this paper](https://doi.org/10.1016/0167-8191(93)90031-F), together with [this paper](https://doi.org/10.1016/0898-1221(93)90109-9) also by P. Amodio. The corresponding code can be found under CR_v1/cyclic_reduction_v8 (yes it took 8 versions to complete). If you want to test the code, good luck. Some tests for various problem sizes are provided, but you'll have to navigate the code yourself. I recently moved some files to clean the repository so running the aforementioned script will most likely fail, but it shouldn't be too hard to fix.

Currently, work is focused on impementing parallel cyclic reduction for the solution of lower block bidiagonal linear systems. This implementation is self made, so it will be interesting to see how it turns out. Most of the work is complete, with only the final step (back-substitution) unfinished. Hopefully, this won't take too long, inshallah. 

